In [2]:
import pandas as pd
pred = pd.read_csv("/Users/christianhellum/Cand. Merc./Data-Science-Project/data_science_project/Beat the bookies/data/df_prepro.csv", index_col=0)

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [13]:
train = pred[pred["date"] < '2024-01-01']

In [14]:
test = matches[matches["date"] > '2024-01-01']

In [15]:
matches.columns

Index(['Unnamed: 0', 'date', 'time', 'round', 'day', 'venue', 'result', 'gf',
       'ga', 'opponent', 'xg', 'xga', 'poss', 'attendance', 'captain',
       'formation', 'referee', 'match report', 'sh', 'sot', 'sot%', 'dist',
       'fk', 'pk', 'pkatt', 'season', 'team', 'touches', 'att 3rd', 'att pen',
       'totdist', 'prgdist', 'prgc', 'cpa', 'rec', 'prgr', 'venue_code',
       'opp_code', 'hour', 'day_code', 'target'],
      dtype='object')

In [16]:
predictors = ["venue_code", "opp_code", "hour", "day_code", 'touches', 'att 3rd', 'att pen']

In [17]:
rf.fit(train[predictors], train["target"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [18]:
preds = rf.predict(test[predictors])

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
error = accuracy_score(test["target"], preds)

In [21]:
error

0.63125

In [22]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [24]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,162,36
1,82,40


In [25]:
from sklearn.metrics import precision_score

precision_score(test["target"], preds)

0.5263157894736842

In [26]:
grouped_matches = matches.groupby("team")

In [27]:
group = grouped_matches.get_group("Real Madrid").sort_values("date")

In [28]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [29]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

,Unnamed: 0,date,time,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
2241,3,2020-10-04,16:00,Matchweek 5,Sun,Away,W,2.0,0.0,Levante,...,6,1,1.333333,0.666667,13.666667,4.000000,15.633333,0.333333,0.333333,0.333333
2242,4,2020-10-17,18:30,Matchweek 6,Sat,Home,L,0.0,1.0,Cádiz,...,5,0,2.000000,0.666667,12.666667,3.666667,14.166667,0.333333,0.333333,0.333333
2243,6,2020-10-24,16:00,Matchweek 7,Sat,Away,W,3.0,1.0,Barcelona,...,5,1,1.000000,0.333333,14.000000,3.333333,16.666667,0.000000,0.000000,0.000000
2244,8,2020-10-31,14:00,Matchweek 8,Sat,Home,W,4.0,1.0,Huesca,...,5,1,1.666667,0.666667,13.666667,4.333333,16.900000,0.000000,0.333333,0.333333
2245,10,2020-11-08,21:00,Matchweek 9,Sun,Away,L,1.0,4.0,Valencia,...,6,0,2.333333,1.000000,13.666667,5.333333,17.300000,0.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,41,2024-03-31,21:00,Matchweek 30,Sun,Home,W,2.0,0.0,Athletic Club,...,6,1,3.333333,1.333333,17.333333,9.000000,16.000000,0.333333,0.000000,0.000000
30,43,2024-04-13,18:30,Matchweek 31,Sat,Away,W,1.0,0.0,Mallorca,...,5,1,3.333333,0.666667,17.000000,8.666667,17.066667,0.000000,0.000000,0.000000
31,45,2024-04-21,21:00,Matchweek 32,Sun,Home,W,3.0,2.0,Barcelona,...,6,1,2.333333,0.666667,14.000000,7.000000,19.233333,0.333333,0.000000,0.000000
32,46,2024-04-26,21:00,Matchweek 33,Fri,Away,W,1.0,0.0,Real Sociedad,...,4,1,2.000000,0.666667,12.666667,6.333333,19.400000,0.333333,0.333333,0.333333


In [30]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

/var/folders/8l/n92rnf315xdf87gkyx0s1n840000gn/T/ipykernel_82350/4052147919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [31]:
matches_rolling

Unnamed: 0       date   time         round  day venue result  \
team                                                                            
Alaves     2773           3 2020-09-30  19:00   Matchweek 4  Wed  Away      L   
           2774           4 2020-10-04  14:00   Matchweek 5  Sun  Home      W   
           2775           5 2020-10-18  18:30   Matchweek 6  Sun  Home      L   
           2776           6 2020-10-25  14:00   Matchweek 7  Sun  Away      W   
           2777           7 2020-10-31  21:00   Matchweek 8  Sat  Home      D   
...                     ...        ...    ...           ...  ...   ...    ...   
Villarreal 301           40 2024-04-01  21:00  Matchweek 30  Mon  Home      L   
           302           41 2024-04-14  18:30  Matchweek 31  Sun  Away      D   
           303           42 2024-04-21  16:15  Matchweek 32  Sun  Away      W   
           304           43 2024-04-28  18:30  Matchweek 33  Sun  Home      W   
           305           44 2024-05-05  16:15  Matchweek 34  Sun  Away      L   

                  gf   ga         opponent  ...  day_code  target  gf_rolling  \
team                                        ...                                 
Alaves     2773  1.0  3.0       Villarreal  ...         2       0    0.333333   
           2774  1.0  0.0    Athletic Club  ...         6       1    0.666667   
           2775  0.0  2.0            Elche  ...         6       0    0.666667   
           2776  2.0  0.0       Valladolid  ...         6       1    0.666667   
           2777  1.0  1.0        Barcelona  ...         5       0    1.000000   
...              ...  ...              ...  ...       ...     ...         ...   
Villarreal 301   1.0  2.0  Atlético Madrid  ...         0       0    3.000000   
           302   1.0  1.0    Athletic Club  ...         6       0    1.666667   
           303   2.0  1.0          Almería  ...         6       1    1.000000   
           304   3.0  0.0   Rayo Vallecano  ...         6       1    1.333333   
           305   2.0  3.0       Celta Vigo  ...         6       0    2.000000   

                 ga_rolling sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                         
Alaves     2773    1.000000   7.000000    1.666667    15.900000   0.666667   
           2774    1.666667   7.333333    2.000000    17.400000   0.666667   
           2775    1.000000   8.333333    2.000000    16.766667   0.333333   
           2776    1.666667   9.666667    2.666667    17.666667   0.000000   
           2777    0.666667  11.333333    2.666667    17.533333   0.000000   
...                     ...        ...         ...          ...        ...   
Villarreal 301     1.000000  11.333333    6.666667    13.633333   0.333333   
           302     1.333333   9.666667    4.000000    13.300000   0.000000   
           303     1.000000  10.333333    3.000000    16.066667   0.000000   
           304     1.333333  10.000000    2.000000    17.766667   0.000000   
           305     0.666667  11.333333    3.666667    17.866667   0.000000   

                 pk_rolling  pkatt_rolling  
team                                        
Alaves     2773    0.000000       0.000000  
           2774    0.000000       0.000000  
           2775    0.000000       0.000000  
           2776    0.000000       0.000000  
           2777    0.000000       0.333333  
...                     ...            ...  
Villarreal 301     0.000000       0.333333  
           302     0.000000       0.333333  
           303     0.333333       1.000000  
           304     0.333333       0.666667  
           305     0.333333       0.666667  

[2876 rows x 49 columns]

In [32]:
matches_rolling = matches_rolling.droplevel('team')

In [33]:
matches_rolling

,Unnamed: 0,date,time,round,day,venue,result,gf,ga,opponent,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
2773,3,2020-09-30,19:00,Matchweek 4,Wed,Away,L,1.0,3.0,Villarreal,...,2,0,0.333333,1.000000,7.000000,1.666667,15.900000,0.666667,0.000000,0.000000
2774,4,2020-10-04,14:00,Matchweek 5,Sun,Home,W,1.0,0.0,Athletic Club,...,6,1,0.666667,1.666667,7.333333,2.000000,17.400000,0.666667,0.000000,0.000000
2775,5,2020-10-18,18:30,Matchweek 6,Sun,Home,L,0.0,2.0,Elche,...,6,0,0.666667,1.000000,8.333333,2.000000,16.766667,0.333333,0.000000,0.000000
2776,6,2020-10-25,14:00,Matchweek 7,Sun,Away,W,2.0,0.0,Valladolid,...,6,1,0.666667,1.666667,9.666667,2.666667,17.666667,0.000000,0.000000,0.000000
2777,7,2020-10-31,21:00,Matchweek 8,Sat,Home,D,1.0,1.0,Barcelona,...,5,0,1.000000,0.666667,11.333333,2.666667,17.533333,0.000000,0.000000,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,40,2024-04-01,21:00,Matchweek 30,Mon,Home,L,1.0,2.0,Atlético Madrid,...,0,0,3.000000,1.000000,11.333333,6.666667,13.633333,0.333333,0.000000,0.333333
302,41,2024-04-14,18:30,Matchweek 31,Sun,Away,D,1.0,1.0,Athletic Club,...,6,0,1.666667,1.333333,9.666667,4.000000,13.300000,0.000000,0.000000,0.333333
303,42,2024-04-21,16:15,Matchweek 32,Sun,Away,W,2.0,1.0,Almería,...,6,1,1.000000,1.000000,10.333333,3.000000,16.066667,0.000000,0.333333,1.000000
304,43,2024-04-28,18:30,Matchweek 33,Sun,Home,W,3.0,0.0,Rayo Vallecano,...,6,1,1.333333,1.333333,10.000000,2.000000,17.766667,0.000000,0.333333,0.666667


In [34]:
matches_rolling.index = range(matches_rolling.shape[0])

In [35]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2024-01-01']
    test = data[data["date"] > '2024-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [36]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [37]:
error

0.5466666666666666

In [38]:
combined

,actual,predicted
91,0,0
92,1,0
93,1,0
94,1,0
95,0,1
...,...,...
2871,0,1
2872,0,0
2873,1,0
2874,1,0


In [39]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [40]:
combined.head(10)

,actual,predicted,date,team,opponent,result
91,0,0,2024-01-02,Alaves,Real Sociedad,D
92,1,0,2024-01-12,Alaves,Sevilla,W
93,1,0,2024-01-19,Alaves,Cádiz,W
94,1,0,2024-01-26,Alaves,Almería,W
95,0,1,2024-02-03,Alaves,Barcelona,L
96,0,0,2024-02-10,Alaves,Villarreal,D
97,0,0,2024-02-18,Alaves,Betis,D
98,0,1,2024-02-24,Alaves,Mallorca,D
99,0,0,2024-03-04,Alaves,Osasuna,L
100,1,1,2024-03-10,Alaves,Rayo Vallecano,W


In [ ]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)